# Athlete Dashboard - Preparation
Development sandbox for the athlete dashboard user interface.

In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import panel as pn
import param
import hvplot as hv

In [2]:
full_rounds = pd.read_csv('../data/scraped/cleaned/rounds_splits.csv')

# replace 0 positions and laptimes with NaN
pos_cols = [f'lap_{x}_position' for x in range(1, 46)]
laptime_cols = [f'lap_{x}_laptime' for x in range(1, 46)]
full_rounds[pos_cols] = full_rounds[pos_cols].replace(0.0, np.nan)
full_rounds[laptime_cols] = full_rounds[laptime_cols].replace(0.0, np.nan)

C:\Users\ahale\Anaconda3\envs\shorttrack\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,19,20,22,23,25,26,28,29,31,34,37,40,43,46,47,49,50,52,53,55,56,58,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,109,110,113,116,119,122,125,128,131,134,137,140,143,146,149,151,152) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
individual_events = full_rounds[full_rounds['event'].isin({'500m', '1000m', '1500m'})]

In [14]:
class AthleteProfileDashboard(param.Parameterized):

    # widgets containing selectors for the user to refine which data they are seeing
    athlete_name = param.ObjectSelector(default=individual_events.iloc[0]['Name'], objects=list(individual_events['Name'].unique()))
    event = param.ObjectSelector(default='All', objects=list(individual_events['event'].unique()) + ['All'])

    def get_athlete_races(self):
        athlete_races = individual_events[individual_events['Name'] == self.athlete_name]
        if self.event == 'All':
            return athlete_races
        else:
            return athlete_races[athlete_races['event'] == self.event]

    def first_lap_positions(self):
        plot = plt.figure()
        plot.suptitle('Most Common Position to Start the Race')
        plot.add_subplot(111).hist(self.get_athlete_races()['lap_1_position'])
        plt.xlabel('Position')
        plt.ylabel('Frequency')
        plt.close(plot)
        return plot

# create an instance of the class
apd = AthleteProfileDashboard(name='Athlete Profile')

# define the dashboard elements using a subset of the rd class plots
dashboard = pn.Column(apd.param, apd.first_lap_positions)

# display the dashboard
dashboard.show()

Launching server at http://localhost:62315
